In [ ]:
# !conda install -n arcgis -q -y sqlalchemy psycopg2 python-slugify geopandas conda-forge::geoalchemy2

In [ ]:
# import arcgis
import logging
import geopandas as gpd
import pandas as pd

from datetime import datetime
from pathlib import Path
from slugify import slugify
from sqlalchemy import create_engine, text, \
    Table, MetaData, Column, \
    Integer, String, BigInteger, Boolean, \
    Float, DateTime, Numeric, \
    UniqueConstraint, CheckConstraint, ForeignKeyConstraint
from geoalchemy2 import Geometry

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Coletar os dados brutos e salvar em Parquet

## Informações de conexão de origem

In [ ]:
with open("./dsn/source.txt") as r:
    src_engine = create_engine(r.read())

## Informações de template do Parquet

In [ ]:
def create_filename(schema, table, extension, item, suffix="", output_path="./data/"):
    today = datetime.now().strftime("%Y/%m/%d")
    filename = f"{schema}_{table}_{suffix}".strip("_")
    extension = extension.strip().lower()

    if extension not in ("csv", "parquet"):
        raise Exception("Apenas CSV ou parquet")
    
    if suffix and not suffix.startswith("_"):
        suffix = "_" + str(suffix)
    
    _file = Path(output_path) / today / item / f"{filename}.{extension}"
    
    if not _file.exists():
        _file.parent.mkdir(parents=True, exist_ok=True)

    return  _file

## Infomações da tabela de origem

In [ ]:
# Argumentos para rodar o pipeline
src_table_name = "resultados_analiticos_mineralometria"
src_schema = "geoq_valida"
src_primary_key = "objectid"

# Coordenadas
src_x_field = "longitude"
src_y_field = "latitude"
src_srid = 4674

# Temporal
src_ts_field = "data_de_analise"
src_ts_format = "%d/%m/%Y"

table_item = "mineralometria"

f"{src_schema}.{src_table_name}"

### Ler tabela de origem e salvar em Parquet

In [ ]:
# Puxar os dados e converter em dataframe
try: 
    df = (
        pd.read_sql_table(
            src_table_name, src_engine, 
            schema=src_schema, index_col=src_primary_key
        )
            # Sanitizar colunas, para não inserir valores fora de padrão na DDL do banco (acentos, caracteres especiais, etc)
            .rename(columns=lambda col: slugify(col, separator="_"))
    )
    
except Exception as e:
    logging.error(e)

assert not df.empty, "Esta tabela está vazia. Abortando..."
assert df.index.is_unique, f"Esta tabela não possui identifcador único de registro no campo <{primary_key}>"

# Gravar em Parquet
out_raw_file = create_filename(src_schema, src_table_name, "parquet", table_item, suffix="raw")
df.to_parquet(out_raw_file, index=True)

# Info
df.info()

# Tratamento dos dados brutos e salvar em PostgreSQL

## Informações de conexão de destino

In [ ]:
dst_schema = "digeop"

with open("./dsn/destiny-geobancao.txt") as r:
    dst_engine = create_engine(r.read(), plugins=["geoalchemy2"])

metadata = MetaData(schema=dst_schema)

## Tabela de amostras (survey)

### Análise de colunas

In [ ]:
# Construção das colunas
pk_column = Column(src_primary_key, Integer(), primary_key=True, autoincrement=False)
geom_column = Column('geometry', Geometry('POINT', srid=src_srid, dimension=2), nullable=False)

fixed_columns = (
    Column("projeto_amostragem", String(255), nullable=False),
    Column("projeto_publicacao", String(255), nullable=False),
    Column("centro_de_custo", String(10), nullable=False),
    Column("classe", String(50), nullable=False),
    Column("numero_de_campo", String(20), nullable=False),
    Column("numero_de_laboratorio", String(8), nullable=True), # Null em rocha
    Column("duplicata", Boolean(), nullable=False, default=False, server_default="0"),
    Column(src_x_field, Float(), nullable=False),
    Column(src_y_field, Float(), nullable=False),
    Column("laboratorio", String(255), nullable=False),
    Column("job", String(30), nullable=True),
    Column("data_de_analise", DateTime(), nullable=True),
    Column("abertura", String(100), nullable=True),
    Column("leitura", String(100), nullable=True), # Null em Mineral de minério
    Column("observacao", String(1024), nullable=True),    
)

# Colunas extras
if table_item in ("mineral_minerio", "rocha"):
    extra_columns = (
        Column("classificacao_petrografica", String(255), nullable=True),
        Column("unidade_litoestratigrafica", String(255), nullable=True),
    ) 
else:
    extra_columns = ()


# Juntando as colunas comuns ao dataframe
survey_columns = fixed_columns[:-2] + extra_columns + fixed_columns[-2:]
survey_column_names = tuple([col.name for col in survey_columns])

# Verificação se a coluna existe no DataFrame
for col in survey_column_names:
    assert col in df.columns, f"The fixed column <{col}> is not in table columns."

# Adicionar pk e geometria na tupla final de colunas
survey_columns = (
    pk_column,
    *survey_columns, 
    geom_column
)
# survey_columns

### Criação do objeto SQL Table

In [ ]:
survey_table_name = f"{table_item}_amostras"

survey_tbl = Table(
    survey_table_name,
    metadata,
    *survey_columns,
    extend_existing=True  # !!!!
)

# survey_tbl

### Extração de dados de amostras

In [ ]:
# GeoPandas
survey_df = gpd.GeoDataFrame(
    df.filter(survey_column_names)
        .apply(lambda col: col.replace("", None))
        .rename(columns=lambda col: slugify(col, separator="_"))    
        .assign(
            # forçar duplicata como booleano
            duplicata=lambda df: df.duplicata.fillna("não").str.match("(sim|1)", case=False)
        ),
    geometry=gpd.points_from_xy(
        df[src_x_field], 
        df[src_y_field], 
        crs=src_srid
    )
)

# Consertar capos de timestamp
if src_ts_field in survey_df.columns:
    data_analise_ser = pd.to_datetime(survey_df[src_ts_field], format=src_ts_format, errors='coerce')

    date_invalid_idx = (
        survey_df[[src_ts_field]]
            .join(
                data_analise_ser,
                rsuffix='_converted'
            )
            .loc[
                lambda df:df.data_de_analise_converted.isna()
            ].index
    )

    if not date_invalid_idx.empty:
        out_date_invalid_file = create_filename(src_schema, src_table_name, "parquet", table_item, suffix="survey_date_invalid")
        survey_df.loc[date_invalid_idx].to_parquet(out_date_invalid_file, index=True)
    
    survey_df[src_ts_field] = data_analise_ser
        
    del data_analise_ser

# ObjectID tem que ser único e não pode ter geometria nula ou vazia
assert survey_df.index.is_unique, f"ObjectID precisa ser único: {survey_df[survey_df.index.duplicated()].index.tolist()}"
assert not (survey_df.geometry.isna().all() and survey_df.geometry.is_empty.all()), "A tabela não pode ter geometria nula ou vazia"

# Gravar em Parquet
out_survey_file = create_filename(src_schema, src_table_name, "parquet", table_item, suffix="survey")
survey_df.to_parquet(out_survey_file, index=True)

# Info
survey_df.info()

## Resultados

### Análise de colunas

In [ ]:
### Colunas a excluir do processo
assay_excluded_columns = ("globalid", "lote", "ra", "metodo", "created_user", "created_date", "last_edited_user", "last_edited_date")

# Colunas a serem pivotadas
assay_columns = [col for col in df.columns if col not in (survey_column_names + assay_excluded_columns)]

# for col in assay_columns_fixed:
#     assert col.name in df.columns, f"The fixed column <{col}> is not in assay columns."

### Tabelas de Pesagens

In [ ]:
weight_sample_column = Column("amostra", Integer(), nullable=False)
weight_analyte_column = Column("analito", String(50), nullable=False)
weight_unit_column = Column("unidade", String(6), nullable=False)
weight_value_column =  Column("valor", Numeric(), nullable=False)


weight_pivoted_columns = (
    Column("id", BigInteger(), primary_key=True, autoincrement=True),
    weight_sample_column,
    # *weight_columns_fixed,
    weight_analyte_column,
    # weight_unit_column,
    # weight_qualif_column,
    weight_value_column
)

weight_table_name = f"{table_item}_info_pesagem"

# Criação da tabela
weight_tbl = Table(
    weight_table_name,
    metadata,
    *weight_pivoted_columns,
    UniqueConstraint(
        weight_sample_column.name, 
        weight_analyte_column.name,
        name=f"{weight_table_name}_uniq"
    ),
    ForeignKeyConstraint(
        [weight_sample_column.name],
        [f"{survey_table_name}.{src_primary_key}"],
        onupdate="RESTRICT",
        ondelete="CASCADE",
    ),
    extend_existing=True  # !!!!
)

weight_tbl

### Tabela de Resultados

In [ ]:
assay_sample_column = Column("amostra", Integer(), nullable=False)
assay_analyte_column = Column("analito", String(50), nullable=False)
assay_unit_column = Column("unidade", String(6), nullable=False)
# assay_qualif_column = Column("qualificador", String(1), nullable=True)
assay_value_column =  Column("valor", String(3), nullable=False)


assay_pivoted_columns = (
    Column("id", BigInteger(), primary_key=True, autoincrement=True),
    assay_sample_column,
    # *assay_columns_fixed,
    assay_analyte_column,
    # assay_unit_column,
    # assay_qualif_column,
    assay_value_column
)

assay_table_name = f"{table_item}_resultados"

# Criação da tabela
assay_tbl = Table(
    assay_table_name,
    metadata,
    *assay_pivoted_columns,
    UniqueConstraint(
        assay_sample_column.name, 
        # "abertura", "leitura", 
        assay_analyte_column.name, # "unidade", 
        name=f"{assay_table_name}_uniq"
    ),
    # TODO: Relacionar com o dicionário de valores enum nas células abaixo
    # CheckConstraint(f"{assay_qualif_column.name} IN (('S1', 'S3', 'S15', 'S40', 'S60', 'S85', 'X', 'Y', 'Z')", name=f"{assay_table_name}_valor_chk"),
    ForeignKeyConstraint(
        [assay_sample_column.name],
        [f"{survey_table_name}.{src_primary_key}"],
        onupdate="RESTRICT",
        ondelete="CASCADE",
    ),
    extend_existing=True  # !!!!
)

assay_tbl

### Extração de dados de análises

In [ ]:
# padrão para achar espaços
spaces_regex = r"\s+"
# Segunda lista é o padrão para missings de read_csv do pandas
missing_values = ["", "N.A."] + [" ", "#N/A", "#N/A N/A", "#NA", "-1.#IND", "-1.#QNAN", "-NaN", "-nan", "1.#IND", "1.#QNAN", "<NA>", "N/A", "NA", "NULL", "NaN", "None", "n/a", "nan", "null"]
normalize_values = {
    ',': "."
}

# Funções para Pipes
def handle_missing(series, extra_missing_values=[]):
    return (
        series.str.replace(spaces_regex, "", regex=True)
            .replace(missing_values + extra_missing_values, None) 
    )

def handle_normalized(series, extra_replaces={}):
    extra_replaces.update(normalize_values)
    
    for key, value in extra_replaces.items():
        series = series.str.replace(key, value)
    return series

# Primeiras limpezas
index_names = [col.name for col in (assay_sample_column, assay_analyte_column)]
value_name = assay_value_column.name

### Dados de pesagem

In [ ]:
# Colunas de peso de amostra
weight_columns = [
    'peso_inicial_g', 'peso_fracionado_g', 'peso_conc_0_3_g', 'peso_bateia_0_3_g',
    'magnetico_ima_mao_g', 'total_pesado_g', 'nao_analisadas_leves_g', 
    'semi_magnetico_g', 'nao_magneticos_g', 'total_analisado_g'
]
    
for col in weight_columns:
    assert col in weight_columns, f"A coluna <{col}> não existe nas colunas de peso de amostra"

# Primeiras limpezas
weight_df = (
    df.filter(weight_columns)
        .apply(lambda col: col.replace("", None))
        .rename(columns=lambda col: slugify(col, separator="_"))
        # De-pivot
        .stack()
        # Ajusta nomes
        .rename_axis(index_names)
        .rename(value_name)
        # handle missing data on values
        .pipe(handle_missing)
        .dropna()
        # normalize values + qualificators
        .pipe(handle_normalized, normalize_values)
        # converte para float (peso não tem qualificador)
        .astype(float)
        # .to_frame()
)

# Index tem que ser único
assert weight_df.index.is_unique

# Gravar em Parquet
#out_weight_file = template_parquet_file.format(schema, table, "weight")
#weight_df.to_parquet(out_weight_file, index=True)

weight_df

### Dados de minerais

In [ ]:
# Metadado de análise
assay_meta = ("leitura",)
assay_analyte_field = "substancia"

for col in assay_meta:
    assert col in assay_meta, f"A coluna <{col}> não existe nas colunas de metadados de análise"

# Colunas com object_id + Assays - (fixed_columns + excluded_columns - colunas de peso)
_excluded = survey_column_names + assay_excluded_columns + tuple(weight_columns)
assay_columns = [col for col in df.columns if col not in _excluded]
del _excluded

# normalizar
extra_values = {
    "1" : "S1",
    "3" : "S3",
    "15": "S15",
    "40": "S40",
    "60": "S60",
    "85": "S85",
}

# Regex para validar valores
semiquant_regex = "^S(1|3|15|40|60|85)$"
qualit_regex = "^(X|Y|Z)$"
    
# Primeiras limpezas
assay_df = (
    df.filter(assay_columns)
        .apply(lambda col: col.replace("", None))
        .rename(columns=lambda col: slugify(col, separator="_"))
        # Traz o objectid para o index do dataframe
        # .set_index(assay_meta, append=True)
        # De-pivot
        .stack()
        # Ajusta nomes
        .rename_axis([assay_sample_column.name] + [assay_analyte_column.name])
        .rename(assay_value_column.name)
        # handle missing data on values
        .pipe(handle_missing)
        .dropna()
        # ajustar valores semiquantitativos
        .replace(extra_values.keys(), extra_values.values())
)

# ObjectID tem que ser único
assert survey_df.index.is_unique

# Valores precisam atender aos padrões de valores
values_match = assay_df[assay_df.str.match(semiquant_regex) | assay_df.str.match(qualit_regex)]

try:
    assert values_match.all(), f"Alguns valores <{assay_df[~values_match].shape[0]}> não coincidiram com a expressão regular: \n{assay_df[~values_match].head()}"

except AssertionError as e:
    logging.error(e)
    assay_error_oids = assay_df[~values_match].index.get_level_values(0).drop_duplicates().tolist()
    df[df.index.isin(assay_error_oids)].to_csv(template_parquet_file.format(schema, table, "errors").replace(".parquet", ".csv"))


# Substituição dos códigos pelos labels
# value_enum = {
#     "S1" : "<1%",
#     "S3" : "1-5%",
#     "S15": "5-25%",
#     "S40": "25-50%",
#     "S60": "50-75%",
#     "S85": ">75%",
#     "X"  : ">50%",
#     "Y"  : "5-50%",
#     "Z"  : "<5%",
# }

# assay_df = assay_df.replace(value_enum).to_frame()

# # Gravar em Parquet
# # out_assay_file = template_parquet_file.format(schema, table, "assay")
# # assay_df.to_parquet(out_assay_file, index=True)

assay_df.head()

# Mandar para o banco de dados de destino

### Criar estruturas de SQL usando o SQLALchemy

In [ ]:
# # Destrói as duas tabelas
# metadata.drop_all(dst_engine)
# metadata.create_all(dst_engine)

### Gravar do Parquet para o banco de dados

In [ ]:
values_match

In [ ]:
# Identificar amostras sem análises inválidas
#valid_samples = assay_df[values_match].index.get_level_values(assay_sample_column.name).drop_duplicates().tolist()

# Gravar o parquet no banco de dados
with dst_engine.connect() as conn:
    with conn.begin():
        # 'to_postgis' não funciona com method. Ver como fazer isso com 'to_sql' tradicional
        logging.info("Gravando amostras...")
        ( #survey_df[survey_df.index.isin(valid_samples)]
         survey_df
            .to_postgis(
                survey_table_name, 
                conn, 
                if_exists='append', 
                schema=dst_schema, 
                index=True, 
                # chunksize=5000
            )
        )

        logging.info("Gravando dados de pesagem...")
        (weight_df
            #.loc[values_match]
            .reset_index()
            .rename_axis(weight_pivoted_columns[0].name)
            .to_sql(
                weight_table_name, 
                conn, 
                if_exists='append', 
                schema=dst_schema, 
                index=False, 
                # chunksize=10000, 
                # method="multi" # https://pandas.pydata.org/docs/user_guide/io.html#io-sql-method
            )
        )

        logging.info("Gravando análises...")
        (assay_df
            #.loc[values_match]
            .reset_index()
            .rename_axis(assay_pivoted_columns[0].name)
            .to_sql(
                assay_table_name, 
                conn, 
                if_exists='append', 
                schema=dst_schema, 
                index=False, 
                # chunksize=10000, 
                # method="multi" # https://pandas.pydata.org/docs/user_guide/io.html#io-sql-method
            )
        )
        
        conn.commit()
        
    logging.info("Reindexando...")
    for tbl in [survey_table_name, assay_table_name, weight_table_name]:
        conn.execute(text(f"REINDEX TABLE {tbl};"))
        
    logging.info("Finalizado!")